In [14]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, FloatType

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag
import string
import re
from pyspark.sql import SparkSession
from textblob import TextBlob



In [15]:
spark = SparkSession \
    .builder \
    .appName("Python Spark Sentiment Analysis example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


In [16]:
df = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true').\
            load("../twitter/stream_covid-19.csv",header=True);

In [22]:
print (df.count())
df.show(5)

338798
+-------------------+--------------+-------------------+---------------+-----+--------------------+--------------------+
|         twitter_id|          name|         created_at|followers_count|place|                text|      text_processed|
+-------------------+--------------+-------------------+---------------+-----+--------------------+--------------------+
|1247396599128420353|logicalindianz|2020-04-07 05:31:39|           5453| null|'RT @ANI: My neig...|my neighbours hav...|
|1247396603029151746| N8V_Calgarian|2020-04-07 05:31:40|           5164| null|'RT @siiamhamilto...|the rcmp surround...|
|1247396604534874112|      kk131066|2020-04-07 05:31:40|          15830| null|'RT @kk131066: Th...|the brains of gop...|
|1247396608116957184|     drjulie_b|2020-04-07 05:31:41|           9315| null|'These R0 numbers...|these numbers are...|
|1247396611526987777|        DArm91|2020-04-07 05:31:42|           5669| null|'RT @WuTangClan: ...|we got you just m...|
+-------------------+----

In [18]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
pat3 = r'^.RT'
hashtag = r'\#\S*'
combined_pat = r'|'.join((pat1, pat2, pat3))
print(combined_pat)
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z19]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()
text_process_udf = udf(tweet_cleaner_updated, StringType())


@[A-Za-z0-9_]+|https?://[^ ]+|^.RT


In [ ]:
def sentiment_analysis(text):
    return TextBlob(text).sentiment.polarity
sentiment_analysis_udf = udf(sentiment_analysis , FloatType())

def condition(r):
    if (r >=0.1):
        label = "positive"
    elif(r <= -0.1):
        label = "negative"
    else:
        label = "neutral"
    return label

sentiment_udf = udf(lambda x: condition(x), StringType())

In [ ]:
%time
df = df.withColumn('text_processed',text_process_udf(df['text']))
df  = df.withColumn("sentiment_score", sentiment_analysis_udf( df['text_processed'] ))
df  = df.withColumn("sentiment", sentiment_udf( df['sentiment_score'] ))
df.show(5,True)

In [13]:
spark.stop()
